The baseline model provided by organizers was really tricky to work with. In particular, it was hard even to re-save the model, since the variables were not recorded.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
model = tf.saved_model.load('../input/baseline-landmark-retrieval-model/baseline_landmark_retrieval_model')

In [ ]:
len(model.variables)

It turned out that getting to model's variables (which then can be used, for example, for loading them into Keras reimplementation, using Keras `set_weights()` function from `Model` API) is tricky. Below I present the way to load the weights into numpy arrays, in order to be able to use them later (for example in Keras).

Firstly, this magic one-liner shows us list of variables that are hidden in model's graph.

In [ ]:
model.graph.get_collection('variables')[:10]

In [ ]:
var_names = [var.name for var in model.graph.get_collection('variables')]

Now that we have variable names, we can use the following trick, which I discovered by reading TF source code and examining model in Tensorboard, to load the weights into numpy arrays.

In [ ]:
test_in = tf.constant(np.uint8(np.random.randn(300, 300, 3)))

In [ ]:
var_names_to_fetch = [
    var_name[:-2] + '/read' + var_name[-2:] for var_name in var_names]

weight_fetcher = model.prune(
    feeds=["input_image:0"],
    fetches=var_names_to_fetch
)

weights = weight_fetcher(test_in)

Now, all numpy arrays with model parameters are saved into eager Tensors in the `weights` list.

In [ ]:
len(weights) == len(model.graph.get_collection('variables'))

In [ ]:
weights[0].numpy().shape, weights[0].numpy()